# LAB | Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A reader model to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

Let's get started by installing the packages needed for notebook to run:

In [ ]:
!pip install python-dotenv

In [ ]:
import pandas as pd
import openai
import os
import warnings

from dotenv import load_dotenv, find_dotenv
warnings.filterwarnings('ignore')

In [ ]:
load_dotenv('/content/env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

print("✅ OPENAI Key loaded:", OPENAI_API_KEY is not None)
print("✅ HuggingFace Key loaded:", HUGGINGFACEHUB_API_TOKEN is not None)
print("✅ Pinecone Key loaded:", PINECONE_API_KEY is not None)

✅ OPENAI Key loaded: True
✅ HuggingFace Key loaded: True
✅ Pinecone Key loaded: True


# Install Dependencies

In [ ]:
!pip install -qU datasets pinecone-client sentence-transformers torch

# Load Dataset

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [ ]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

In [ ]:
df.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [ ]:
# Select only the 'title' and 'context' columns
df = df[['title', 'context']]

# Count duplicated rows based on 'context' only
print("Duplicate context count:", df.duplicated(subset=['context']).sum())

# Count duplicated rows based on 'title' only
print("Duplicate title count:", df.duplicated(subset=['title']).sum())

Duplicate context count: 68708
Duplicate title count: 87157


In [ ]:
# Drop rows containing duplicate context passages
df = df.drop_duplicates(subset=['context'])

# Display the final dataframe
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
87574,Kathmandu,"Institute of Medicine, the central college of ..."
87579,Kathmandu,Football and Cricket are the most popular spor...
87584,Kathmandu,The total length of roads in Nepal is recorded...
87589,Kathmandu,The main international airport serving Kathman...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)

Now we create a new index called "question-answering" — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [ ]:
!pip install pinecone

In [ ]:
from pinecone import PineconeApiException

index_name = "question-answering"

try:
    # Try to create the index
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=spec
    )
    print(f"Index '{index_name}' created.")
except PineconeApiException as e:
    if "ALREADY_EXISTS" in str(e):
        print(f"Index '{index_name}' already exists. Skipping creation.")
    else:
        raise e  # re-raise any other errors

# Connect to the index
index = pc.Index(index_name)


Index 'question-answering' created.


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the retriever model from HuggingFace
retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)

# Print the retriever model to confirm it's loaded
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

In [ ]:
from tqdm.auto import tqdm

# We'll use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # 1. Determine the end index of the current batch
    end = min(i + batch_size, len(df))

    # 2. Extract the current batch from the dataframe
    batch = df.iloc[i:end]

    # 3. Generate embeddings using the retriever model
    emb = retriever.encode(batch['context'].tolist(), show_progress_bar=False)

    meta = [{'title': row['title'], 'context': row['context']} for _, row in batch.iterrows()]

    # 5. Create unique IDs for each item
    ids = [f"doc-{j}" for j in range(i, end)]

    # 6. Combine everything into a list of tuples (id, vector, metadata)
    to_upsert = list(zip(ids, emb, meta))

    # 7. Upsert the batch into the Pinecone index
    _ = index.upsert(vectors=to_upsert)

# Finally, check the index statistics
index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891,
 'vector_type': 'dense'}

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [ ]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering', device=device)
reader

Device set to use cuda


Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [ ]:
# Gets context passages from the Pinecone index
def get_context(question, top_k=5):
    # 1. Generate the embedding vector for the input question
    xq = retriever.encode([question])[0]  # [0] to get the actual vector, not a list

    # Convert xq to a native Python list
    xq = xq.tolist()

    # 2. Query the Pinecone index to find the top_k most relevant context passages
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)

    # 3. Extract the context passages from the results
    contexts = [match['metadata']['context'] for match in xc['matches']]

    return contexts

In [ ]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [ ]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k=1)
print(context[0])

The economy is a mixture of village agriculture and handicrafts, an industrial sector based largely on petroleum, support services, and a government characterized by budget problems and overstaffing. Petroleum extraction has supplanted forestry as the mainstay of the economy. In 2008, oil sector accounted for 65% of the GDP, 85% of government revenue, and 92% of exports. The country also has large untapped mineral wealth.


As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [ ]:
extract_answer(question, context)

[{'answer': '92% of exports',
  'context': 'The economy is a mixture of village agriculture and handicrafts, '
             'an industrial sector based largely on petroleum, support '
             'services, and a government characterized by budget problems and '
             'overstaffing. Petroleum extraction has supplanted forestry as '
             'the mainstay of the economy. In 2008, oil sector accounted for '
             '65% of the GDP, 85% of government revenue, and 92% of exports. '
             'The country also has large untapped mineral wealth.',
  'end': 372,
  'score': 9.097009024117142e-05,
  'start': 358}]


The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [ ]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Sir Tim Berners-Lee',
  'context': 'The first web browser was invented in 1990 by Sir Tim '
             'Berners-Lee. Berners-Lee is the director of the World Wide Web '
             "Consortium (W3C), which oversees the Web's continued "
             'development, and is also the founder of the World Wide Web '
             'Foundation. His browser was called WorldWideWeb and later '
             'renamed Nexus.',
  'end': 65,
  'score': 5.350271441162402e-12,
  'start': 46}]


In [ ]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity '
             'and general relativity. He also made important contributions to '
             'statistical mechanics, especially his mathematical treatment of '
             'Brownian motion, his resolution of the paradox of specific '
             'heats, and his connection of fluctuations and dissipation. '
             'Despite his reservations about its interpretation, Einstein also '
             'made contributions to quantum mechanics and, indirectly, quantum '
             'field theory, primarily through his theoretical studies of the '
             'photon.',
  'end': 86,
  'score': 0.9500371813774109,
  'start': 29}]


Let's run another question. This time for top 3 context passages from the retriever.

In [ ]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

The result looks pretty good.

In [ ]:
#pc.delete_index(index_name)

### Add a few more questions. What did you observe?

In [ ]:
question = "Why do cats purr, and do they purr when they are not happy?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'normal droppings, which are not eaten',
  'context': 'Other animals, such as rabbits and rodents, practise coprophagia '
             'behaviours - eating specialised faeces in order to re-digest '
             'food, especially in the case of roughage. Capybara, rabbits, '
             'hamsters and other related species do not have a complex '
             'digestive system as do, for example, ruminants. Instead they '
             'extract more nutrition from grass by giving their food a second '
             'pass through the gut. Soft faecal pellets of partially digested '
             'food are excreted and generally consumed immediately. They also '
             'produce normal droppings, which are not eaten.',
  'end': 542,
  'score': 1.0116795579051541e-07,
  'start': 505}]


In [50]:
question = "Can plants hear or respond to sound?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Plants also communicate via volatiles',
  'context': 'Communication is observed within the plant organism, i.e. within '
             'plant cells and between plant cells, between plants of the same '
             'or related species, and between plants and non-plant organisms, '
             'especially in the root zone. Plant roots communicate with '
             'rhizome bacteria, fungi, and insects within the soil. These '
             'interactions are governed by syntactic, pragmatic, and semantic '
             'rules,[citation needed] and are possible because of the '
             'decentralized "nervous system" of plants. The original meaning '
             'of the word "neuron" in Greek is "vegetable fiber" and recent '
             'research has shown that most of the microorganism plant '
             'communication processes are neuron-like. Plants also communicate '
             'via volatiles when exposed to herbivory attack behavior, thus '
             'wa

In [52]:
question = "Is time slower on a plane compared to Earth’s surface?"
context = get_context(question, top_k=4)
extract_answer(question, context)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'answer': 'significantly out of sync with standard working hours',
  'context': 'The manipulation of time at higher latitudes (for example '
             'Iceland, Nunavut or Alaska) has little impact on daily life, '
             'because the length of day and night changes more extremely '
             'throughout the seasons (in comparison to other latitudes), and '
             'thus sunrise and sunset times are significantly out of sync with '
             'standard working hours regardless of manipulations of the clock. '
             'DST is also of little use for locations near the equator, '
             'because these regions see only a small variation in daylight in '
             'the course of the year.',
  'end': 328,
  'score': 5.488159331434872e-07,
  'start': 275},
 {'answer': 'can it be travelled',
  'context': 'Two contrasting viewpoints on time divide many prominent '
             'philosophers. One view is that time is part of the fundamental '
             'stru

# ***What did i observe?***
##**During testing, I observed that setting top_k controls how many context passages are retrieved from the Pinecone index. Each passage is then passed to a reader model which scores potential answers. The model returns higher scores when the context is more semantically relevant to the question. Therefore, the overall performance depends on both the quality of embeddings and the data stored in the vector database**.

